# Building a Retrieval Augmented Generation (RAG) Chatbot

In [52]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
!pip install -qU langchain-mistralai
!pip install -qU langchain-chroma

## Setup

In [55]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for Langsmith: ")

Enter API key for Langsmith:  ········


## Components

In [84]:
import getpass
import os

# Chat model

MODEL_NAME = "mistral-small-latest"
if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral API: ")

Enter API key for Mistral API:  ········


In [86]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

# Chat model initialization
llm = ChatMistralAI(
    mistral_api_key=MISTRAL_API_KEY,
    model="mistral-small-latest",
    streaming=True,
    temperature=0.6
)

# Embeddings model
embeddings = MistralAIEmbeddings(model="mistral-embed")

C:\Users\Laura GALINDO\anaconda3\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [87]:
import chromadb
from chromadb import PersistentClient
from langchain.vectorstores import Chroma

# 1️⃣ Connect to the existing ChromaDB instance
db_path = "./final_chroma_db"
collection_name = "final_vector_store_collection"

client = PersistentClient(path=db_path)
collection = client.get_or_create_collection(name=collection_name)
print(f"✅ Connected to ChromaDB collection '{collection_name}' in '{db_path}'.")

# 2️⃣ Load stored embeddings from ChromaDB
stored_data = collection.get()

# 3️⃣ Check how many embeddings are loaded
num_loaded = len(stored_data["ids"])
print(f"🔍 Loaded {num_loaded} embeddings from ChromaDB.")


# 🔄 Recharger le vector store depuis la base de données ChromaDB
vector_store = Chroma(
    persist_directory=db_path,
    collection_name=collection_name,
    embedding_function=embeddings
)

# ✅ Vérification du nombre de documents chargés
num_loaded = vector_store._collection.count()
print(f"🔍 Vector store loaded with {num_loaded} documents.")

✅ Connected to ChromaDB collection 'final_vector_store_collection' in './final_chroma_db'.
🔍 Loaded 104750 embeddings from ChromaDB.
🔍 Vector store loaded with 104750 documents.


## LLM Prompt-Engineering

In this part, we implement an advanced AI chatbot using LangChain and LangGraph.
It integrates :
- RAG (Retrieval-Augmented Generation) with source display
- Streaming responses for enhanced user experience
- Conversational memory for tracking interactions

In [90]:
#  Importation des bibliothèques
import os
import json
import getpass
import requests

from langchain_mistralai.chat_models import ChatMistralAI
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompts import PipelinePromptTemplate

### Prompt definition

In [93]:
prompt_principal = ChatPromptTemplate.from_template(
    """
    Tu es **Crystal Bot**, un assistant intelligent d'orientation professionnelle.
    Tu dois fournir des réponses basées sur les informations récupérées.
    
    Ton objectif est d'aider chaque utilisateur à **clarifier son avenir** en lui proposant **des conseils sur-mesure et motivants**.

    **Contexte pertinent issu de la base de connaissances :** 
    {context}
    
    ➤ **Adapte-toi à l’utilisateur** : si indécis, propose des pistes d’exploration.
    ➤ **Si comparaison de plusieurs options**, aide-le à peser le pour et le contre.
    ➤ **Propose une réponse fluide, naturelle et pas trop longue**, qui ne se contente pas d’une liste figée, mais qui établit un dialogue et donne des solutions plus large.
    """
)

topic_prompt = PromptTemplate.from_template(
    """
    Analyse la question suivante et identifie le ou les domaines principaux auxquels elle appartient.

    Question: {input}

    Réponds en listant **un ou plusieurs domaines** parmi les suivants (séparés par une virgule) :
    - "Informatique et Numérique"
    - "Sciences et Ingénierie"
    - "Santé et Médecine"
    - "Droit et Sciences Politiques"
    - "Économie et Gestion"
    - "Sciences Sociales et Psychologie"
    - "Arts, Design et Culture"
    - "Lettres et Langues"
    - "Sciences de l'Éducation"
    - "Communication et Journalisme"
    - "Tourisme, Hôtellerie et Restauration"
    - "Environnement et Développement Durable"
    - "Sport et Activité Physique"
    - "Architecture et Urbanisme"
    - "Agronomie et Agroalimentaire"

    Si aucun domaine n’est clair, réponds uniquement par "Autre".

    Domaines identifiés :
    """
)


intent_prompt = PromptTemplate.from_template(
    """
    Analyse la question suivante et identifie l’intention principale de l’utilisateur :

    Question: {input}

    Réponds par l’une ou plusieurs des options suivantes :
    - "Métiers"
    - "Formations"
    - "Salaire"
    - "Offres d'emploi"
    - "Parcoursup"
    - "Autre"

    Intention détectée :
    """
)


chatbot_prompt = PromptTemplate.from_template(
    """
    
    Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨  
    
    L'utilisateur a demandé : {input}.
    Domaine détecté : {topic} (caché à l'utilisateur)
    Intention détectée : {intent} (caché à l'utilisateur)
    
  ➤ **Si l'intention est "Métiers"** :
        - Propose des **métiers adaptés** au profil de l’utilisateur en allant au-delà des évidences.   
        - Explique **les missions, conditions de travail et débouchés** en t’adaptant à la curiosité de l’utilisateur.  
        - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  
        - **Fournis une vision inspirante et dynamique**, et pas seulement une simple liste de métiers.

    ➤ **Si l'intention est "Formations"** :
        - Présente des **formations pertinentes** mais **ajoute des alternatives originales** (double cursus, parcours atypiques).
        - Décris les options de manière dynamique : **BTS, Licence, Master, écoles spécialisées, mais aussi MOOC et certifications**.  
        - **Varie les approches selon la question de l’utilisateur**, sans donner toujours la même liste figée.
        - Rassure l’utilisateur en expliquant que *sa motivation est un facteur clé*. *Donne lui des conseils*
        - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée. 
        - Si pertinent propose à l'utilisateur de visiter des salons pour en savoir plus sur les formations. 

    ➤ **Si l'intention est "Salaire"** :
        - Donne **un aperçu détaillé des salaires**, mais va au-delà des chiffres standards en ajoutant **les tendances du marché**.  
        - Explique **les différences selon l’expérience, le secteur, et la localisation**.  
        - **Si pertinent, parle des avantages indirects** (avantages en nature, perspectives d’évolution, travail à l’international).  
         - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  

    ➤ **Si l'intention est "Offres d'emploi"** :
        - **Décris le marché actuel** du secteur concerné : quelles entreprises recrutent ? Quelles compétences sont recherchées ?
        - Donne **des conseils personnalisés** pour optimiser les candidatures, en fonction du domaine visé.
        - **Propose une approche proactive** : comment créer des opportunités, réseauter efficacement ou se démarquer ?  
         - **N’hésite pas à raconter une anecdote ou une tendance actuelle, personnalise la réponse** pour rendre la réponse plus vivante et personnalisée.  

    ➤ **Si l'intention est "Parcoursup"** :
        - **Explique Parcoursup en t’adaptant au profil de l’utilisateur** (lycéen stressé, étudiant en réorientation…).
        - Fournis **des conseils concrets pour réussir son dossier** et éviter les pièges.  
        - Fournis **des conseils concrets pour réussir la lettre de motivation** et se distinguer des autres candidats. 
        - Si pertinent, parle **des alternatives à Parcoursup** (écoles privées, études à l’étranger, admissions parallèles).  
    

    ➤ **Si l'intention est "Autre"** :
        - **Interprète la demande de l’utilisateur avec souplesse** et propose une réponse originale.  
        - Si possible, **ouvre la discussion sur des idées nouvelles** en fonction de son profil.
        - **Demande lui plus de précisions sur ce qu'il souhaite apprendre ou approfondir** en terme professionnel ou de formation.
         
    """
)


closing_prompt = PromptTemplate.from_template(
    """
    J’espère que cette réponse **t’a aidé à y voir plus clair !** 🔮  
    **As-tu d’autres questions ?** Je suis là pour explorer toutes les possibilités avec toi.  
    """
)

### Setting up conversation memory

In [96]:
# Mise en place de la mémoire conversationnelle
memory = MemorySaver()
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    """Appelle le modèle avec historique de conversation"""
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
chatbot = workflow.compile(checkpointer=memory)

## Creating the retrieval chain with LangChain

Creating a retrieval chain that, based on a user question, will retrieve relevant Documents from Chroma and provide them in context to the MistralAI language model to generate an informed answer

In [99]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

def build_retrieval_chain(vector_store, llm, k=5):
    """
    Build a retrieval chain using a given vector store and LLM
    
    Parameters:
        - vector_store: The vector store containing document embeddings
        - llm: The language model to generate responses
        - k: Number of most relevant document chunks to retrieve (default is 5)
    
    Returns:
        - retrieval_chain: A LangChain retrieval chain object.
    """
    # Create a retriever from the vector store
    retriever = vector_store.as_retriever(search_kwargs={"k": k})
    
    # Load a pre-defined prompt template for retrieval-based Q&A
    retrieval_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
    
    # Create a document combination chain
    combine_docs_chain = create_stuff_documents_chain(llm, retrieval_chat_prompt)
    
    # Create and return the retrieval chain
    return create_retrieval_chain(retriever, combine_docs_chain)

def retrieve_and_format_context(user_query, vector_store, llm):
    """Retrieve documents from vector store and format the context"""
    retrieval_chain = build_retrieval_chain(vector_store, llm)
    retrieved_docs = retrieval_chain.invoke({"input": user_query})

    documents = retrieved_docs["context"]

    formatted_context = "\n".join([doc.page_content for doc in documents])
    sources = [doc.metadata.get("source", "Unknown source") for doc in documents]

    return formatted_context, sources

# Building a full ChatBot response

In [102]:
def generate_response(user_query, vector_store, llm):
    """Generate an optimized response with CrystalBot and keep the history"""

    context, sources = retrieve_and_format_context(user_query, vector_store, llm)
    
    global conversation_history  # Utilisation d'une variable distincte pour stocker les messages

    # Charger l'historique des messages de manière locale
    past_messages = conversation_history if 'conversation_history' in globals() else []  

    # **🔹 Étape 1 : Détection du domaine / topic**
    topic_output = topic_prompt.format(input=user_query)

    # **🔹 Étape 2 : Détection de l’intention utilisateur**
    intent_output = intent_prompt.format(input=user_query)
    if not intent_output:
        intent_output = "Autre"

    # **🔹 Étape 3 : Génération de la réponse principale**
    chatbot_prompt_hidden = chatbot_prompt.format(
        input=user_query, 
        topic=topic_output, 
        intent=intent_output,
        context=context,
    )

    chatbot_output = chatbot_prompt_hidden.replace(topic_output, "").replace(intent_output, "")

    # **🔹 Étape 4 : Génération des suggestions d’exploration selon l’intention détectée**
    exploration_suggestions = ""

    if "Métiers" in intent_output:
        exploration_suggestions += """
        **🌍 Pour aller plus loin :**  
        - 🔍 **Découvre des métiers similaires** via des plateformes comme Onisep ou Studyrama.  
        - 🎭 **Participe à des salons professionnels** et rencontres avec des experts du domaine.  
        - 👥 **Échange avec des professionnels** sur LinkedIn ou lors d’événements.  
        """

    if "Formations" in intent_output or "Parcoursup" in intent_output:
        exploration_suggestions += """
        **📚 Pour approfondir ton parcours :**  
        - 🎤 **Découvre les parcours d’anciens étudiants** via des témoignages en ligne.  
        - 🔎 **Explore les formations en alternance** et internationales.  
        - 🏫 **Assiste aux journées portes ouvertes** des écoles et universités.  
        """

    if "Salaire" in intent_output:
        exploration_suggestions += """
        **💰 Pour mieux comprendre les salaires et évolutions de carrière :**  
        - 📊 **Consulte des études de rémunération** sur Glassdoor et l’APEC.  
        - 📈 **Analyse les évolutions de carrière possibles** en fonction de ton secteur.  
        """

    if "Offres d'emploi" in intent_output:
        exploration_suggestions += """
        **📝 Pour trouver des opportunités professionnelles :**  
        - 🔍 **Consulte des plateformes spécialisées** comme Indeed ou Pôle Emploi.  
        - 📩 **Optimise ton CV et ta lettre de motivation** pour te démarquer.  
        """

    if "Les deux" in intent_output:
        exploration_suggestions += """
        **🌟 Pour lier formations et métiers :**  
        - 📚 **Découvre les formations qui recrutent le plus**.  
        - 👀 **Explore les tendances de recrutement dans ton secteur**.  
        """

    if "Autre" in intent_output:
        exploration_suggestions += """
        **🔍 Explorons d’autres pistes !**  
        - 🤔 **Précise un peu plus ta demande**, veux-tu parler de reconversion, d’entrepreneuriat, d’études à l’étranger ?  
        - 💡 **Inspiration** : Parfois, explorer d’autres secteurs peut ouvrir des portes inattendues.  
        - 🔎 **Découvre des parcours inspirants** : interviews, conférences, podcasts sur des choix de carrière atypiques.  
        """

    # **🔹 Étape 5 : Conclusion dynamique**
    closing_output = closing_prompt.format()

    # ✅ **Ajout d’une introduction engageante pour la toute première réponse**
    intro_message = f"""
    Bonjour ! 😊  

    Je suis **Crystal Bot**, ton conseiller d'orientation professionnelle. 🔮✨  
    Mon objectif est de **t’aider à clarifier ton avenir** en te proposant des pistes adaptées et personnalisées.  
    Voici quelques idées et conseils pour t’aider à avancer :  
    """

    # ✅ **Création du message final à envoyer au modèle**
    final_prompt = f"""
    {intro_message}  

    **💡 Recommandation de Crystal Bot :**  
    {chatbot_output}  

    {exploration_suggestions}  

    {closing_output}  
    """

    # ✅ **Ajout du message dans l'historique**
    messages = past_messages + [HumanMessage(content=final_prompt)]

    # ✅ **Envoi de la requête complète au modèle**
    response = llm.invoke(messages)

    # ✅ **Mise à jour de l'historique de la conversation**
    conversation_history = messages + [response]

    return {
        "response": response.content,
        "sources": sources
    }

def refine_response(chatbot_output, user_query, llm):
    """Improve clarity and precision of response with LLM"""
    refinement_prompt = f"""
    Améliore la réponse suivante en la rendant plus précise et engageante :
    Contexte : {user_query}
    Réponse initiale : {chatbot_output}
    Réponse améliorée :
    """

    refined_response = llm.invoke([HumanMessage(content=refinement_prompt)]).content

    return refined_response

## Testing the ChatBot

In [117]:
user_query = "Quels métiers sont accessibles après un diplôme en informatique ?"

# Tester la récupération des documents depuis ChromaDB
context, sources = retrieve_and_format_context(user_query, vector_store, llm)

print("🔹 Contexte récupéré :")
print(context)
print("\n🔹 Sources associées :")
print(sources)

🔹 Contexte récupéré :
{"annee": 2019, "diplome": "MASTER LMD", "numero_de_l_etablissement": "0331766R", "etablissement": "Bordeaux - Montaigne", "etablissementactuel": null, "code_de_l_academie": "A04", "academie": "Bordeaux", "code_du_domaine": "LLA", "domaine": "Lettres, langues, arts", "code_de_la_discipline": "disc06", "discipline": "Lettres, langues, arts", "situation": "18 mois apr\u00e8s le dipl\u00f4me",
"80", "emplois_stables": "52", "emplois_a_temps_plein": "88", "salaire_net_median_des_emplois_a_temps_plein": "1360", "salaire_brut_annuel_estime": "21200", "de_diplomes_boursiers": "42", "taux_de_chomage_regional": "6.6", "salaire_net_mensuel_median_regional": "1820", "emplois_cadre": "52", "emplois_exterieurs_a_la_region_de_luniversite": "46", "femmes": "90",
"discipline": "Histoire-g\u00e9ographie", "situation": "30 mois apr\u00e8s le dipl\u00f4me", "remarque": null, "nombre_de_reponses": 47, "taux_de_reponse": "86", "poids_de_la_discipline": "14", "taux_dinsertion": "85", "

In [123]:
# Tester la réponse initiale générée par le chatbot
initial_result = generate_response(user_query, vector_store, llm)

# 🔥 Appel de refine_response pour améliorer la réponse
refined_result = refine_response(initial_result["response"], user_query, llm)

print("\n🔹 Réponse affinée du chatbot :")
print(refined_result)


🔹 Réponse affinée du chatbot :
Salut ! 😊

Un diplôme en informatique ouvre la porte à une multitude de métiers passionnants et en pleine expansion. Voici quelques métiers accessibles après un diplôme en informatique, avec des missions, conditions de travail et débouchés détaillés :

### 1. **Développeur Web/Full Stack**
   - **Missions** : Concevoir, développer et maintenir des sites web et des applications web. Utiliser des technologies front-end (HTML, CSS, JavaScript) et back-end (Python, Java, Node.js).
   - **Conditions de travail** : Souvent en entreprise, en agence web ou en freelance. Les horaires peuvent être flexibles, mais les deadlines peuvent être serrées.
   - **Débouchés** : Évolution vers des postes de chef de projet, architecte logiciel ou responsable technique.
   - **Anecdote** : Le développement web est un domaine en constante évolution. Par exemple, l'utilisation de frameworks comme React et Vue.js a révolutionné la manière dont les applications web sont construit

In [107]:
user_query = "hello, quels métiers conseilles-tu à quelqu'un qui aime le contact avec les gens ?"

# Tester la récupération des documents depuis ChromaDB
context, sources = retrieve_and_format_context(user_query, vector_store, llm)

# Tester la réponse initiale générée par le chatbot
initial_result = generate_response(user_query, vector_store, llm)

# 🔥 Appel de refine_response pour améliorer la réponse
refined_result = refine_response(initial_result["response"], user_query, llm)

print(refined_result)

Salut ! 😊

Tu aimes le contact avec les gens ? Génial ! Il existe de nombreux métiers qui te permettront de mettre à profit cette qualité. Voici quelques suggestions qui pourraient vraiment te plaire :

1. **Conseiller en Vente** :
   - **Missions** : Accueillir les clients, comprendre leurs besoins et les conseiller sur les produits ou services les plus adaptés.
   - **Conditions de travail** : Souvent en magasin ou en ligne, avec des horaires variables selon les besoins de l'entreprise.
   - **Débouchés** : Possibilité d'évoluer vers des postes de manager de vente ou de responsable commercial.
   - **Anecdote** : De plus en plus de conseillers en vente utilisent des outils numériques pour offrir une expérience client personnalisée, comme des applications de réalité augmentée pour visualiser les produits.

2. **Responsable des Ressources Humaines (RH)** :
   - **Missions** : Gérer le recrutement, la formation, la gestion des carrières et le bien-être des employés.
   - **Conditions de

In [109]:
user_query = "Je souhaite intégrer une CPGE scientifique. Peux-tu m'aider à rédiger ma lettre de motivation ?"

# Tester la récupération des documents depuis ChromaDB
context, sources = retrieve_and_format_context(user_query, vector_store, llm)

# Tester la réponse initiale générée par le chatbot
initial_result = generate_response(user_query, vector_store, llm)

# 🔥 Appel de refine_response pour améliorer la réponse
refined_result = refine_response(initial_result["response"], user_query, llm)

print(refined_result)

Bien sûr, je serais ravi de t'aider à rédiger une lettre de motivation percutante pour intégrer une CPGE scientifique. Voici un exemple de lettre de motivation que tu peux adapter en fonction de ton parcours et de tes aspirations :

---

**[Ton Prénom et Nom]**
[Ton Adresse]
[Code Postal, Ville]
[Ton Email]
[Ton Numéro de Téléphone]

**[Nom de l'École ou de l'Université]**
[Adresse de l'École ou de l'Université]
[Code Postal, Ville]

**[Date]**

Objet : Candidature pour intégrer une CPGE scientifique

Madame, Monsieur,

Actuellement élève en terminale [spécialité], je souhaite ardemment intégrer votre CPGE scientifique afin de préparer au mieux mon avenir académique et professionnel. Passionné(e) par les sciences et les mathématiques, je suis convaincu(e) que votre établissement me permettra de développer mes compétences et de réaliser mes ambitions.

Depuis mon plus jeune âge, la curiosité m'a poussé(e) à explorer les mystères de notre univers. Cette fascination pour les phénomènes na

In [113]:
sources

['C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json',
 'C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json',
 'C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json',
 'C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json',
 'C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json']